### Importando Bibliotecas

In [71]:
# Processamento de dados
import pandas as pd
import csv
import cv2

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# Webscraping
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from urllib.parse import urljoin

# Outros
import threading
import easyocr
import re

import warnings
warnings.filterwarnings('ignore')

### Salvando todos os links em arquivo txt

In [72]:
# Acessa o site de docentes do DC e salva todos os links da página em um arquivo txt
url = 'https://site.dc.ufscar.br/docentes'

# Configurar o webdriver do Selenium
driver = webdriver.Chrome()  # Certifique-se de ter o ChromeDriver instalado e no PATH

# Fazer uma solicitação HTTP para a URL usando o Selenium
driver.get(url)

# Obter o conteúdo da página após o carregamento dinâmico
page_content = driver.page_source

# Usar BeautifulSoup para analisar o HTML
soup = BeautifulSoup(page_content, 'html.parser')

# Encontrar todos os links na página
links = [a['href'] for a in soup.find_all('a', href=True)]

# Escrever os links em um arquivo txt
with open('links.txt', 'w') as f:
    for link in links:
        f.write("%s\n" % link)

# Fechar o webdriver do Selenium
driver.quit()


### Filtrando links

In [73]:
# abrir o arquivo links.txt
with open('links.txt', 'r') as f:
    links = f.readlines()

# Filtrar os links
docentes_links = []

# se o link contém 'docente' e não contém 'docentes', adicionar à lista docentes_links
for link in links:
    if 'docente' in link and 'docentes' not in link:
        docentes_links.append(link)

# escrever os links filtrados em um arquivo txt
with open('docentes_links.txt', 'w') as f:
    for link in docentes_links:
        f.write("%s" % link)


print(f'Os links foram salvos no arquivo "docentes_links.txt".')


Os links foram salvos no arquivo "docentes_links.txt".


### Acesso a cada link para pegar informações

In [74]:
# Configurar o webdriver do Selenium
driver = webdriver.Chrome()  # Certifique-se de ter o ChromeDriver instalado e no PATH

# abrir o arquivo docentes_links.txt
with open('docentes_links.txt', 'r') as txt_file:
    docentes_links = [row.strip() for row in txt_file]

url = 'https://site.dc.ufscar.br'

nomesEmails = []

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# Escrever os dados dos docentes em um arquivo CSV
with open('docentes.csv', 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Nome', 'Telefone', 'Lattes', 'Email'])  # Escrever o cabeçalho

    for link in docentes_links:
        # Fazer uma solicitação HTTP para a URL usando o Selenium
        driver.get(url + link)

        # Obter o conteúdo da página após o carregamento dinâmico
        page_content = driver.page_source

        # Usar BeautifulSoup para analisar o HTML
        soup = BeautifulSoup(page_content, 'html.parser')

        # obter o nome do docente (o nome está dentro de uma div com id "docente-conteudo" e dentro de uma tag h3)
        nome = soup.find('div', id='docente-conteudo').find('h3').text

        # obter o telefone do docente (o telefone está dentro de uma div com id "docente-conteudo" e dentro da quarta tag p dentro da div)
        telefone = soup.find('div', id='docente-conteudo').find_all('p')[3].text

        # obter o lattes do docente (o lattes está dentro de uma div com id "docente-conteudo" e dentro de uma tag a)
        lattes = soup.find('div', id='docente-conteudo').find('a', href=re.compile(r'lattes')).get('href')

        # obter o email do docente (o email está dentro de uma div com id "docente-conteudo", dentro da terceira tag p e é uma imagem)
        email = soup.find('div', id='docente-conteudo').find_all('p')[2].find('img').get('src')

        nomesEmails.append(email)

        # Escrever os dados do docente no arquivo CSV
        csv_writer.writerow([nome, telefone, lattes, ''])

# Fechar o webdriver do Selenium
driver.quit()

print(f'Os dados dos docentes foram salvos no arquivo "docentes.csv".')

Os dados dos docentes foram salvos no arquivo "docentes.csv".


### Limpando CSV

In [75]:
# ler o arquivo docentes.csv
docentes = pd.read_csv('docentes.csv')

# substituir o texto "Telefone: " por "" (vazio)
docentes['Telefone'] = docentes['Telefone'].str.replace('Telefone: ', '')

# salvar o arquivo docentes.csv
docentes.to_csv('docentes.csv', index=False)

print(f'Os dados dos docentes foram salvos no arquivo "docentes.csv".')

Os dados dos docentes foram salvos no arquivo "docentes.csv".


### Baixando imagens

In [76]:
def baixar_imagem(url):
    resposta = requests.get(url, headers=headers)
    nome = url.split('/')[-1]
    destino = '/home/luis/Documentos/GitHub/Informacoes-Professores/emails/' + nome
    with open(destino, 'wb') as f:
        f.write(resposta.content)

linksEmails = []

for link in nomesEmails:
    linksEmails.append(urljoin(url, link))

threads = []
for url in linksEmails:
    thread = threading.Thread(target=baixar_imagem, args=(url,))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

['https://site.dc.ufscar.br/emails/alanvalejo.png', 'https://site.dc.ufscar.br/emails/alexandre.levada.png', 'https://site.dc.ufscar.br/emails/andrebackes.png', 'https://site.dc.ufscar.br/emails/andreendo.png', 'https://site.dc.ufscar.br/emails/auri.png', 'https://site.dc.ufscar.br/emails/comin.png', 'https://site.dc.ufscar.br/emails/daniel.lucredio.png', 'https://site.dc.ufscar.br/emails/delano.png', 'https://site.dc.ufscar.br/emails/kato.png', 'https://site.dc.ufscar.br/emails/ednaldo.png', 'https://site.dc.ufscar.br/emails/emerson.png', 'https://site.dc.ufscar.br/emails/fcferrari.png', 'https://site.dc.ufscar.br/emails/fredy.png', 'https://site.dc.ufscar.br/emails/helenacaseli.png', 'https://site.dc.ufscar.br/emails/helio.png', 'https://site.dc.ufscar.br/emails/heloisa.png', 'https://site.dc.ufscar.br/emails/hermes.png', 'https://site.dc.ufscar.br/emails/jander.png', 'https://site.dc.ufscar.br/emails/joice.png', 'https://site.dc.ufscar.br/emails/vivaldini.png', 'https://site.dc.ufsc

### Limpando nomes emails

In [77]:
# no array nomesEmails, remova a string /emails/ e a extensão .png de cada elemento
nomesEmails = [re.sub(r'\/emails\/|\.png', '', nomeEmail) for nomeEmail in nomesEmails]

print(nomesEmails)

['alanvalejo', 'alexandre.levada', 'andrebackes', 'andreendo', 'auri', 'comin', 'daniel.lucredio', 'delano', 'kato', 'ednaldo', 'emerson', 'fcferrari', 'fredy', 'helenacaseli', 'helio', 'heloisa', 'hermes', 'jander', 'joice', 'vivaldini', 'luciano', 'marcela', 'marcio', 'marilde.santos', 'mauricio', 'murillo', 'naldi', 'felice', 'orides', 'matias', 'pedrobugatti', 'priscilasaito', 'renato', 'cerri', 'rferrari', 'menotti', 'rrc', 'ferrari', 'rsinoue', 'sandra', 'zorzo', 'valtervcamargo', 'vania', 'desouza']


### Utilizando ML para verificar o nome dos emails nas imagens

In [78]:
# pegue todas imagens da pasta emails, converta para escala de cinza, obtenha o texto da imagem e armazene na coluna Email do arquivo docentes.csv
# ler o arquivo docentes.csv
docentes = pd.read_csv('docentes.csv')

# criar uma lista com os nomes dos arquivos
nomes = []

for nome in nomesEmails:
    nomes.append(nome + '.png')

# criar uma lista com os emails dos arquivos
emails = []

for nome in nomes:
    reader = easyocr.Reader(['en'])

    # Carregar a imagem
    imagem = cv2.imread('/home/luis/Documentos/GitHub/Informacoes-Professores/emails/' + nome)

    # Converter para escala de cinza
    gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)

    result = reader.readtext(gray)

    for detection in result:
        texto = detection[1]

    emails.append(texto)



Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


['alanvalejo.png', 'alexandre.levada.png', 'andrebackes.png', 'andreendo.png', 'auri.png', 'comin.png', 'daniel.lucredio.png', 'delano.png', 'kato.png', 'ednaldo.png', 'emerson.png', 'fcferrari.png', 'fredy.png', 'helenacaseli.png', 'helio.png', 'heloisa.png', 'hermes.png', 'jander.png', 'joice.png', 'vivaldini.png', 'luciano.png', 'marcela.png', 'marcio.png', 'marilde.santos.png', 'mauricio.png', 'murillo.png', 'naldi.png', 'felice.png', 'orides.png', 'matias.png', 'pedrobugatti.png', 'priscilasaito.png', 'renato.png', 'cerri.png', 'rferrari.png', 'menotti.png', 'rrc.png', 'ferrari.png', 'rsinoue.png', 'sandra.png', 'zorzo.png', 'valtervcamargo.png', 'vania.png', 'desouza.png']


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster wi

### Formatando emails

Como o modelo de Machine Learning não é exatamente preciso, ele pode cometer erros. Então, já que nós já temos os nomes dos emails (parte antes do @) por conta do 'src' das imagens, basta verificar se a quantidade de caracteres após o @ será mais que 9 (len('ufscar.br') == 9). Isso porque o modelo pode trocar caracteres, mas é extremamente difícil ele omitir caracteres. Se a quantidade for maior que 9, significa que após o @ será dc.ufscar.br, senão, será ufscar.br

In [79]:
# no array emails, conte quantos caracteres tem após o caractere @ em cada elemento
# se o número de caracteres for maior que 9, a variável ufscar  = False, senão, ufscar = True
ufscar = []

for email in emails:
    if len(email[email.find('@') + 1:]) > 9:
        ufscar.append(False)
    else:
        ufscar.append(True)

for i in range(len(emails)):
    if ufscar[i]:
        ufscar[i] = nomesEmails[i] + '@ufscar.br'
    else:
        ufscar[i] = nomesEmails[i] + '@dc.ufscar.br'

# substituir a coluna Email do arquivo docentes.csv pela lista ufscar
docentes['Email'] = ufscar

# salvar o arquivo docentes.csv
docentes.to_csv('docentes.csv', index=False)

print(f'Os emails dos docentes foram salvos no arquivo "docentes.csv".')


Os emails dos docentes foram salvos no arquivo "docentes.csv".
